# Courses Demo
This Jupyter notebook is for exploring the data set courses20-21.json
which consists of all Brandeis courses in the 20-21 academic year (Fall20, Spr21, Sum21) 
which had at least 1 student enrolled.

First we need to read the json file into a list of Python dictionaries

In [1]:
import json

In [2]:
with open("courses20-21.json","r",encoding='utf-8') as jsonfile:
    courses = json.load(jsonfile)

## Structure of a course
Next we look at the fields of each course dictionary and their values

In [3]:
print('there are',len(courses),'courses in the dataset')
print('here is the data for course 1246')
courses[1246]

there are 7813 courses in the dataset
here is the data for course 1246


{'limit': 28,
 'times': [{'start': 1080, 'end': 1170, 'days': ['w', 'm']}],
 'enrolled': 4,
 'details': 'Instruction for this course will be offered remotely. Meeting times for this course are listed in the schedule of classes (in ET).',
 'type': 'section',
 'status_text': 'Open',
 'section': '1',
 'waiting': 0,
 'instructor': ['An', 'Huang', 'anhuang@brandeis.edu'],
 'coinstructors': [],
 'code': ['MATH', '223A'],
 'subject': 'MATH',
 'coursenum': '223A',
 'name': 'Lie Algebras: Representation Theory',
 'independent_study': False,
 'term': '1203',
 'description': "Theorems of Engel and Lie. Semisimple Lie algebras, Cartan's criterion. Universal enveloping algebras, PBW theorem, Serre's construction. Representation theory. Other topics as time permits. Usually offered every second year.\nAn Huang"}

## Cleaning the data
If we want to sort courses by instructor or by code, we need to replace the lists with tuples (which are immutable lists)

In [4]:
for course in courses:
        course['instructor'] = tuple(course['instructor'])
        course['coinstructors'] = tuple([tuple(f) for f in course['coinstructors']])
        course['code']= tuple(course['code'])

In [5]:
print('notice that the instructor and code are tuples now')
courses[1246]

notice that the instructor and code are tuples now


{'limit': 28,
 'times': [{'start': 1080, 'end': 1170, 'days': ['w', 'm']}],
 'enrolled': 4,
 'details': 'Instruction for this course will be offered remotely. Meeting times for this course are listed in the schedule of classes (in ET).',
 'type': 'section',
 'status_text': 'Open',
 'section': '1',
 'waiting': 0,
 'instructor': ('An', 'Huang', 'anhuang@brandeis.edu'),
 'coinstructors': (),
 'code': ('MATH', '223A'),
 'subject': 'MATH',
 'coursenum': '223A',
 'name': 'Lie Algebras: Representation Theory',
 'independent_study': False,
 'term': '1203',
 'description': "Theorems of Engel and Lie. Semisimple Lie algebras, Cartan's criterion. Universal enveloping algebras, PBW theorem, Serre's construction. Representation theory. Other topics as time permits. Usually offered every second year.\nAn Huang"}

# Exploring the data set
Now we will show how to use straight python to explore the data set and answer some interesting questions. Next week we will start learning Pandas/Numpy which are packages that make it easier to explore large dataset efficiently.

Here are some questions we can try to asnwer:
* what are all of the subjects of courses (e.g. COSI, MATH, JAPN, PHIL, ...)
* which terms are represented?
* how many instructors taught at Brandeis last year?
* what were the five largest course sections?
* what were the five largest courses (where we combine sections)?
* which are the five largest subjects measured by number of courses offered?
* which are the five largest courses measured by number of students taught?
* which course had the most sections taught in 20-21?
* who are the top five faculty in terms of number of students taught?
* etc.

In [6]:
## all the subjects of courses

subjects = [course['subject'] for course in courses]
subjects[:5]

['HRNS', 'HRNS', 'HRNS', 'HRNS', 'HRNS']

In [7]:
# terms represented

{course['term'] for course in courses}

{'1203', '1211', '1212'}

In [8]:
#how many instructors taught at brandeis with at least 1 student last year

len({course['instructor'] for course in courses if course['enrolled'] > 0})

808

In [9]:
# display subjects

{course['subject'] for course in courses}

{'AAAS',
 'AAAS/HIS',
 'AAAS/WGS',
 'AAPI',
 'AAPI/HIS',
 'AAPI/WGS',
 'AAS/AAPI',
 'AMST',
 'AMST/ENG',
 'AMST/MUS',
 'ANTH',
 'ANTH/WGS',
 'ARBC',
 'BCBP',
 'BCHM',
 'BIBC',
 'BIOL',
 'BIOT',
 'BIPH',
 'BISC',
 'BUS',
 'BUS/ECON',
 'BUS/FIN',
 'CA',
 'CAST',
 'CBIO',
 'CHEM',
 'CHIN',
 'CHSC',
 'CLAS',
 'CLAS/ENG',
 'CLAS/NEJ',
 'COMH',
 'COML',
 'COMP',
 'COSI',
 'EAS',
 'EBIO',
 'ECON',
 'ECON/FIN',
 'ECS',
 'ECS/ENG',
 'ED',
 'EL',
 'ENG',
 'ENVS',
 'ESL',
 'FA',
 'FILM',
 'FIN',
 'FREN',
 'GECS',
 'GER',
 'GRK',
 'GS',
 'HBRW',
 'HISP',
 'HIST',
 'HIST/SOC',
 'HIST/WGS',
 'HOID',
 'HRNS',
 'HS',
 'HS/POL',
 'HSSP',
 'HUM',
 'HUM/UWS',
 'HWL',
 'IGS',
 'IIM',
 'IMES',
 'INT',
 'ITAL',
 'JAPN',
 'JOUR',
 'KOR',
 'LALS',
 'LAT',
 'LGLS',
 'LING',
 'MATH',
 'MERS',
 'MUS',
 'NBIO',
 'NEJS',
 'NEUR',
 'NPSY',
 'PAX',
 'PHIL',
 'PHYS',
 'PMED',
 'POL',
 'POL/WGS',
 'PSYC',
 'QBIO',
 'QR',
 'RBIF',
 'RBOT',
 'RCOM',
 'RDFT',
 'RDMD',
 'RECS',
 'RECS/THA',
 'REL',
 'RHIN',
 'RIAS',
 'RID

In [10]:
# hello

{course['name'] for course in courses}

{'History of Music II: Late Baroque and Classical',
 'Macroeconomic Theory',
 'Introduction to Cognitive Neuroscience',
 'Principles of Learning Experience Design',
 'Reflective Teaching',
 'Urban Worlds',
 'Continuing Latin',
 'Beginning Yiddish',
 'The Social Fabric: An Anthropology of Fashion',
 'Investigating Justice',
 'Disability Cultures: Art, Film and Literature of People with Disabilities',
 'Introduction to International and Global Studies',
 'Communication for Effective Leadership',
 'Beginning Latin',
 'German Culture Through Film',
 'Blackness and Horror',
 'Introduction to Geographic Information Systems',
 'Exam Preparation',
 'Introduction to Algebra, Part II',
 'Mergers and Acquisitions Analysis',
 'Applied International Macroeconomics',
 'Corporate Financial Engineering',
 'Neurogenetics',
 'Politics on a Pedestal: Statues, Sculpture, Monuments',
 'Studies in Asian Art',
 'Individual Research and Advanced Work',
 'Organizational Leadership and Decision Making',
 'Geogr